# 1. Load packages

The link to get [ImageAnalysis3](https://github.com/zhengpuas47/ImageAnalysis3) 

or the Zhuang lab archived [source_tools](https://github.com/ZhuangLab/Chromatin_Analysis_2020_cell/tree/master/sequential_tracing/source)

In [1]:
%run "C:\Users\shiwei\Documents\ImageAnalysis3\required_files\Startup_py3.py"
sys.path.append(r"C:\Users\shiwei\Documents")

import ImageAnalysis3 as ia
%matplotlib notebook

from ImageAnalysis3 import *
print(os.getpid())

import h5py
from ImageAnalysis3.classes import _allowed_kwds
import ast

import pandas as pd

41452


See **functions** in the repository for [AnalysisTool_Chromatin](../../functions/README.md)

In [2]:
# Chromatin_analysis_tools (ATC)
# Get path for the py containing functions
import os
import sys
import importlib
module_path =r'C:\Users\shiwei\Documents\AnalysisTool_Chromatin'
if module_path not in sys.path:
    sys.path.append(module_path)
    
# import relevant modules
import gene_selection 
importlib.reload(gene_selection)
import gene_to_loci
importlib.reload(gene_to_loci)
import gene_activity
importlib.reload(gene_activity)
import loci_1d_features
importlib.reload(loci_1d_features)  

import atac_to_loci
importlib.reload(atac_to_loci)

<module 'atac_to_loci' from 'C:\\Users\\shiwei\\Documents\\AnalysisTool_Chromatin\\atac_to_loci.py'>

In [3]:
import seaborn as sns
import scanpy as sc
import numpy as np

## 1.1 Define output folder

In [4]:
output_main_folder = r'L:\Shiwei\Figures\MOp_draft_2023_v2\Nuclear_organization_v2'
output_analysis_folder = os.path.join(output_main_folder, 'analysis')
output_figure_folder = os.path.join(output_main_folder, 'figures')

make_output_folder = True

if make_output_folder and not os.path.exists(output_analysis_folder):
    os.makedirs(output_analysis_folder)
    print(f'Generating analysis folder: {output_analysis_folder}.')
elif os.path.exists(output_analysis_folder):
    print(f'Use existing analysis folder: {output_analysis_folder}.')
    
if make_output_folder and not os.path.exists(output_figure_folder):
    os.makedirs(output_figure_folder)
    print(f'Generating figure folder: {output_figure_folder}.')
elif os.path.exists(output_figure_folder):
    print(f'Use existing figure folder: {output_figure_folder}.')


Use existing analysis folder: L:\Shiwei\Figures\MOp_draft_2023_v2\Nuclear_organization_v2\analysis.
Use existing figure folder: L:\Shiwei\Figures\MOp_draft_2023_v2\Nuclear_organization_v2\figures.


## 1.2 setting up plotting parameters

In [5]:
sc.settings.n_jobs = 24
sc.settings.set_figure_params(dpi=150, dpi_save=300, frameon=False, 
                              figsize=(4, 4), fontsize=8, facecolor='white', transparent=True)
sc.settings.figdir = output_figure_folder

from matplotlib import rcParams
rcParams['font.family'] = 'sans-serif'
rcParams['font.sans-serif'] = ['Arial']

# 2. Load codebook and chr2Zxys

## Codebook

Example of [codebook](../../postanalysis/0_locus_annotation/resources/MERFISH_loci_adjacent_genes_tss_more_res.csv)

In [14]:
# load codebook
# L drive is Crick Pu_SSD_0
codebook_save_folder=r'C:\Users\shiwei\Documents\AnalysisTool_Chromatin\resources'

# Load sorted codebook (allows direct matrix slicing) with cell type info
# Load codebook (then sort to allow direct matrix slicing) with cell type info

codebook_fname = os.path.join(codebook_save_folder,'MERFISH_loci_adjacent_genes_tss_more_res.csv')
codebook_df = pd.read_csv (codebook_fname, index_col=0)

# sort df temporailiy so matrix can be sliced by df order directly
codebook_df = loci_1d_features.sort_loci_df_by_chr_order (codebook_df)

codebook_df.head()

,name,id,chr,chr_order,library,dtype,adjacent_genes_0kb_tss,adjacent_genes_10kb_tss,adjacent_genes_50kb_tss,adjacent_genes_100kb_tss,adjacent_genes_200kb_tss,adjacent_genes_500kb_tss,adjacent_genes_1000kb_tss,adjacent_genes_1500kb_tss,adjacent_genes_2000kb_tss
loci_name,,,,,,,,,,,,,,,
chr1_3742742_3759944,1:3742742-3759944,1,1,0,CTP11,combo,intergenic,intergenic,intergenic,Xkr4,Xkr4,Gm18956; Gm7341; Xkr4,Gm18956; Gm2053; Gm6085; Gm6101; Gm6119; Gm734...,Atp6v1h; Gm16041; Gm17101; Gm18956; Gm2053; Gm...,Atp6v1h; Gm16041; Gm17101; Gm18956; Gm2053; Gm...
chr1_6245958_6258969,1:6245958-6258969,2,1,1,CTP11,combo,intergenic,intergenic,4732440D04Rik; Rb1cc1,4732440D04Rik; Rb1cc1,4732440D04Rik; Gm19026; Gm2147; Rb1cc1,4732440D04Rik; Gm19026; Gm2147; Npbwr1; Rb1cc1...,4732440D04Rik; Gm19026; Gm2147; Gm5694; Gm7182...,4732440D04Rik; Atp6v1h; Gm16041; Gm17101; Gm19...,4732440D04Rik; Atp6v1h; Gm16041; Gm17101; Gm19...
chr1_8740008_8759916,1:8740008-8759916,3,1,2,CTP11,combo,intergenic,intergenic,Gm15452,Gm15452,Gm15452,Gm15452,1700034P13Rik; 2610203C22Rik; Adhfe1; Gm15452;...,1700034P13Rik; 2610203C22Rik; Adhfe1; Arfgef1;...,1700034P13Rik; 2610203C22Rik; Adhfe1; Arfgef1;...
chr1_9627926_9637875,1:9627926-9637875,1,1,3,CTP13,combo,2610203C22Rik,2610203C22Rik,2610203C22Rik,2610203C22Rik; Adhfe1; Gm6161; Mybl1; Rrs1,1700034P13Rik; 2610203C22Rik; Adhfe1; Gm18300;...,1700034P13Rik; 2610203C22Rik; Adhfe1; Cops5; C...,1700034P13Rik; 2610203C22Rik; Adhfe1; Arfgef1;...,1700034P13Rik; 2610203C22Rik; Adhfe1; Arfgef1;...,1700034P13Rik; 2610203C22Rik; A830018L16Rik; A...
chr1_9799472_9811359,1:9799472-9811359,2,1,4,CTP13,combo,Gm6195,Gm6195; Sgk3,Gm6195; Sgk3,1700034P13Rik; Gm6195; Mcmdc2; Mybl1; Sgk3; Vc...,1700034P13Rik; 2610203C22Rik; Gm6195; Mcmdc2; ...,1700034P13Rik; 2610203C22Rik; Adhfe1; Arfgef1;...,1700034P13Rik; 2610203C22Rik; Adhfe1; Arfgef1;...,1700034P13Rik; 2610203C22Rik; Adhfe1; Arfgef1;...,1700034P13Rik; 2610203C22Rik; A830018L16Rik; A...


## Chr2Zxys dict and medianDict

Data can be generated from notebook

[preprocess/2_dna_merfish/scripts/2_spot_pick/4_summarize_jie_to_dict](../../preprocess/2_dna_merfish/scripts/2_spot_pick/4_summarize_jie_to_dict.ipynb)

In [7]:
postanalysis_folder = r'\\10.245.74.158\Chromatin_NAS_8\Analyzed_data\MouseBrain_PostAnalysis_20230201\MOp_WT_postanalysis\postanalysis_vCW2_sorted'
subclass_2_chr2Zxys_filename = os.path.join(postanalysis_folder, 'subclass_2_chr2Zxys.pkl')
subclass_2_chr2ZxysList = pickle.load(open(subclass_2_chr2Zxys_filename, 'rb'))

In [8]:
subclass_2_chr2ZxysList.keys()

dict_keys(['Oligo', 'L5 IT', 'Micro', 'Peri', 'Endo', 'Astro', 'OPC', 'L6 CT', 'L5 ET', 'L5/6 NP', 'Pvalb', 'L6 IT', 'Lamp5', 'L6b', 'Sst', 'SMC', 'L4/5 IT', 'L2/3 IT', 'Vip', 'Sncg', 'VLMC'])

In [9]:
# subclass info
subclass_2_cellInfoList_filename = os.path.join(postanalysis_folder, 'subclass_2_cellInfo.pkl')
subclass_2_cellInfoList = pickle.load(open(subclass_2_cellInfoList_filename, 'rb'))

# 3. Calculate radial position for cell (dataframe version)

## Define functions

In [10]:
from scipy.spatial.distance import euclidean, cdist, pdist
from scipy.spatial import ConvexHull

In [11]:
# quick get cell pts as zxy arr
def chrZXY_to_cell_pts_arr (chrZxysList_cell:dict):
    cell_pts = []
    for _chrom, _chr_zxys in chrZxysList_cell.items():
        for _fiberidx, _ichr_zxys in enumerate(_chr_zxys):
            cell_pts.extend(_ichr_zxys)
    return cell_pts
    
# generate cell pts df with region names
from AnalysisTool_Chromatin import loci_1d_features
importlib.reload(loci_1d_features)  

def chrZXY_to_cell_pts_dataframe (chrZxysList_cell:dict, 
                                  codebook_df:pd.core.frame.DataFrame, 
                                  cellgroup = 'None'):
    
    output_df_list = []
    for _chrom, _chr_zxys in chrZxysList_cell.items():
        for _fiberidx, _ichr_zxys in enumerate(_chr_zxys):
            _fiber_df = pd.DataFrame(_ichr_zxys,columns = ['z','x','y'],dtype=np.float64)
            _fiber_df['chr']=_chrom
            _fiber_df['fiberidx']=_fiberidx+1
            # get other info from codebook
            _info_df = codebook_df[codebook_df['chr']==_chrom].copy(deep=True)
            _info_df = loci_1d_features.sort_loci_df_by_chr_order(_info_df)
            _info_df['chr_order'] = _info_df['chr_order'].map(lambda x: int(x))
            _info_df.reset_index(inplace=True) # rest index to match so cols can be assigned
            if len(_info_df) == len(_fiber_df):
                _fiber_df[['name','chr_order']]=_info_df[['name','chr_order']]
            else:
                print ('Number of loci does NOT match between input dict and codebook.')
                #return None
            output_df_list.append(_fiber_df)
    
    output_df = pd.concat(output_df_list)
    output_df['celltype']=cellgroup
    return output_df
                

In [12]:
np.unique(codebook_df['chr'])

array(['1', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19',
       '2', '3', '4', '5', '6', '7', '8', '9', 'X', 'Y'], dtype=object)

In [13]:
# method to get interection radius for normalization
# convert hull center to 0,0,0 first
def get_intersection(U,hull):
    eq=hull.equations.T
    V,b=eq[:-1],eq[-1]
    alpha=-b/np.dot(U,V)
    return U*np.full(np.array(U).shape, np.min(alpha[alpha>0]))

In [14]:
def Radial_position_dataframe (chr_2_zxys_list,
                                  cell_infoList_class,
                                 codebook_df,
                                 #function='nanmedian', axis=0, 
                                #data_num_th=100,  # nonnan data num th for nanmedian along axis
                                #Zxys_num_th=500,  # total nonnan data Zxys for using convex hull 
                                #summarize_result=True,
                                 verbose=False):
    
    from scipy.spatial.distance import euclidean, cdist, pdist
    from scipy.spatial import ConvexHull
    import math
    
    # 1. prepare dict to store loci-2-center dist results
    _out_cell_dict = {'uid': {},
                      'subclass': {},      
                      'total_spots': {},
                      'radial_position':{}
                     }
    # make sure is sorted to match the order and has the loci name as index
    codebook_df = loci_1d_features.sort_loci_df_by_chr_order (codebook_df)
    # loop through cells
    for _cell_idx, _cell_chr_2_zxys_dict in enumerate(chr_2_zxys_list):
        
        _out_cell_dict['uid'][_cell_idx] = cell_infoList_class[_cell_idx]['uid']
        _out_cell_dict['subclass'][_cell_idx] = cell_infoList_class[_cell_idx]['subclass']
        # 1.1. get nuclei centroid and radius
        # 1.1. get nuclei centroid and radius
        combined_cell_chrZxy_list = chrZXY_to_cell_pts_arr(_cell_chr_2_zxys_dict)
        # use all non-nan points to calculate convex hull            
        nonnan_index =np.where(np.sum(~np.isnan(combined_cell_chrZxy_list)>0,axis=1))
        total_spots_num = len(nonnan_index[0])
        _out_cell_dict['total_spots'][_cell_idx] = total_spots_num
        # convex hull
        points=np.array(combined_cell_chrZxy_list)[nonnan_index]
        hull = ConvexHull(points)  
        # get hull centroid
        cz = np.mean(hull.points[hull.vertices,0])
        cx = np.mean(hull.points[hull.vertices,1])
        cy = np.mean(hull.points[hull.vertices,2])
        # attention: zxy in correct order!!
        _hull_ct_zxy = np.array([cz,cx,cy])
        
        # update point and hull
        points = points - _hull_ct_zxy
        hull = ConvexHull(points)

        # 1.2. caluclate normalized dist to nuclei centroid for radial positioning if applicable
        _cell_radial_position_dict = {#'chr': [],
                                      #'chr_order': [],
                                      'loci_name': [], # edit to use chr/chr_order or loci_name depending on the codebook format
                                      'fiber_id': [],
                                      'norm_radial_position': [],}
        # loop through chr
        for _chr_key in np.unique(codebook_df['chr']):
            codebook_df_chr = codebook_df[codebook_df['chr']==_chr_key]
            # it's possible this way, some loci will not exist in the final output, use the codebook to assign nan for these loci if so
            # otherwise, add empty result for each chr
            if _chr_key in _cell_chr_2_zxys_dict.keys(): 
                _chr_zxys = _cell_chr_2_zxys_dict[_chr_key]
                ###########################################
                # calculate normalized radial pos for each fiber
                for _ichr_idx, _ichr_zxys in enumerate(_chr_zxys):
                    ichr_zxys = _ichr_zxys - _hull_ct_zxy 
                    # define output
                    _loci_2_center_list = []
                    _loci_radius_list = []
                    #loop through every point
                    for _zxys in ichr_zxys:
                        if len(np.where(np.isnan(_zxys))[0])>0:
                            _loci_2_center_list.append(np.nan)
                            _loci_radius_list.append(np.nan)
                        else:
                            _loci_2_center_list.append(euclidean(_zxys, [0,0,0]))
                            edge_pt = get_intersection(_zxys, hull)
                            _loci_radius_list.append(euclidean(edge_pt, [0,0,0]))
                    # normalize each loci on the fiber
                    _loci_2_center_list = np.array(_loci_2_center_list)/np.array(_loci_radius_list)
                    # add result
                    _cell_radial_position_dict['loci_name'].extend(codebook_df_chr.index)
                    _cell_radial_position_dict['fiber_id'].extend([_ichr_idx+1]*len(codebook_df_chr))
                    _cell_radial_position_dict['norm_radial_position'].extend(_loci_2_center_list)
                 ##############################################
        _out_cell_dict['radial_position'][_cell_idx] = pd.DataFrame(_cell_radial_position_dict)
        
    _out_cell_df = pd.DataFrame(_out_cell_dict)
    _out_cell_df.set_index('uid', drop=True,inplace=True)
    return _out_cell_df


## Process

In [45]:
_class = 'Peri'
_out_cell_df = Radial_position_dataframe (subclass_2_chr2ZxysList[_class][:10],
                                              subclass_2_cellInfoList[_class][:10],
                                              codebook_df)

_out_cell_df

,subclass,total_spots,radial_position
uid,,,
146535126853720688714614085779839345592,Peri,1423,loci_name fiber_id norm...
125550627244524118102850004242680097855,Peri,172,loci_name fiber_id norm_...
36351220340074970867983016910520020206,Peri,1184,loci_name fiber_id norm...
189925718821537239617372360226094268394,Peri,1348,loci_name fiber_id norm...
336701155824180762964569319502078897172,Peri,1067,loci_name fiber_id norm...
44941500310719804279057173322640311048,Peri,185,loci_name fiber_id norm_...
35537425374731960985039793938329920228,Peri,2971,loci_name fiber_id norm...
286444965641588660267952743112952433756,Peri,866,loci_name fiber_id norm...
134687006417940118526153657302531923221,Peri,922,loci_name fiber_id norm...


In [46]:
_out_cell_df.iloc[0]['radial_position']

,loci_name,fiber_id,norm_radial_position
0,chr1_3742742_3759944,1,NaN
1,chr1_6245958_6258969,1,NaN
2,chr1_8740008_8759916,1,NaN
3,chr1_9627926_9637875,1,0.908119
4,chr1_9799472_9811359,1,NaN
...,...,...,...
2902,chrX_163750534_163758706,1,0.934195
2903,chrX_166247682_166259932,1,NaN
2904,chrX_167157164_167167452,1,0.710625
2905,chrX_168746045_168757590,1,NaN


In [47]:
import tqdm
_out_cell_df_list = []

for _class, chr_2_zxys_list in tqdm.tqdm(subclass_2_chr2ZxysList.items()):
    
    print (f'Analyze cell hull volume for {_class}', end=' ')
    cell_infoList_class = subclass_2_cellInfoList[_class]
    
    _start_time = time.time()
    _out_cell_df = Radial_position_dataframe (chr_2_zxys_list,
                                              cell_infoList_class,
                                              codebook_df)
    _out_cell_df_list.append(_out_cell_df)
    print(f"in {time.time()-_start_time:.3f}s.")

merged_radial_pos_df = pd.concat(_out_cell_df_list)
merged_radial_pos_df


  0%|                                                                                           | 0/21 [00:00<?, ?it/s]

Analyze cell hull volume for Oligo 


  5%|███▊                                                                            | 1/21 [09:05<3:01:54, 545.73s/it]

in 545.729s.
Analyze cell hull volume for L5 IT 


 10%|███████▌                                                                        | 2/21 [12:55<1:54:01, 360.09s/it]

...


 95%|██████████████████████████████████████████████████████████████████████████████    | 20/21 [54:43<01:34, 94.59s/it]

in 10.933s.
Analyze cell hull volume for VLMC 

100%|█████████████████████████████████████████████████████████████████████████████████| 21/21 [55:08<00:00, 157.54s/it]

in 24.858s.


,subclass,total_spots,radial_position
uid,,,
259202492748634617304623818845147108919,Oligo,863,loci_name fiber_id norm...
336074706103419484501535095917410831417,Oligo,1426,loci_name fiber_id norm...
179117357717369278884189379535888871018,Oligo,1601,loci_name fiber_id norm...
36321914521561890358202806077673807953,Oligo,210,loci_name fiber_id norm_...
92271180002801802991585284299263797402,Oligo,2034,loci_name fiber_id norm...
...,...,...,...
89559371323596133331060964655277765256,VLMC,584,loci_name fiber_id norm...
144314319466287448491677682735506973,VLMC,387,loci_name fiber_id norm_...
71119619609696733523981100580894538756,VLMC,734,loci_name fiber_id norm...


In [48]:
merged_radial_pos_df.iloc[0]['radial_position']

,loci_name,fiber_id,norm_radial_position
0,chr1_3742742_3759944,1,0.663333
1,chr1_6245958_6258969,1,NaN
2,chr1_8740008_8759916,1,NaN
3,chr1_9627926_9637875,1,NaN
4,chr1_9799472_9811359,1,0.389658
...,...,...,...
1803,chr9_118751319_118759956,1,NaN
1804,chr9_119425689_119434008,1,0.545641
1805,chr9_121252615_121259973,1,NaN
1806,chr9_123457587_123467505,1,NaN


## Save

In [49]:
# below will compress the radial position df to str; do hdf5 save instead
#output_fname = os.path.join(output_analysis_folder, f'radial_pos_convexhull_bysubclass.csv')
#merged_radial_pos_df.to_csv(output_fname)
output_fname = os.path.join(output_analysis_folder, f'radial_pos_convexhull_bysubclass_v2.h5')
merged_radial_pos_df.to_hdf(output_fname, key='df', mode='w')

C:\Users\shiwei\AppData\Local\Temp\2\ipykernel_31136\3413044594.py:5: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block1_values] [items->Index(['subclass', 'radial_position'], dtype='object')]

  merged_radial_pos_df.to_hdf(output_fname, key='df', mode='w')


## Load

In [6]:
output_fname = os.path.join(output_analysis_folder, f'radial_pos_convexhull_bysubclass_v2.h5')

merged_radial_pos_df = pd.read_hdf(output_fname)
merged_radial_pos_df.head()

,subclass,total_spots,radial_position
uid,,,
259202492748634617304623818845147108919,Oligo,863,loci_name fiber_id norm...
336074706103419484501535095917410831417,Oligo,1426,loci_name fiber_id norm...
179117357717369278884189379535888871018,Oligo,1601,loci_name fiber_id norm...
36321914521561890358202806077673807953,Oligo,210,loci_name fiber_id norm_...
92271180002801802991585284299263797402,Oligo,2034,loci_name fiber_id norm...


In [7]:
merged_radial_pos_df.iloc[0]['radial_position']

,loci_name,fiber_id,norm_radial_position
0,chr1_3742742_3759944,1,0.663333
1,chr1_6245958_6258969,1,NaN
2,chr1_8740008_8759916,1,NaN
3,chr1_9627926_9637875,1,NaN
4,chr1_9799472_9811359,1,0.389658
...,...,...,...
1803,chr9_118751319_118759956,1,NaN
1804,chr9_119425689_119434008,1,0.545641
1805,chr9_121252615_121259973,1,NaN
1806,chr9_123457587_123467505,1,NaN


# 4. Summarize for median of average and Plot

In [10]:
np.unique(merged_radial_pos_df['subclass'])

array(['Astro', 'Endo', 'L2/3 IT', 'L4/5 IT', 'L5 ET', 'L5 IT', 'L5/6 NP',
       'L6 CT', 'L6 IT', 'L6b', 'Lamp5', 'Micro', 'OPC', 'Oligo', 'Peri',
       'Pvalb', 'SMC', 'Sncg', 'Sst', 'VLMC', 'Vip'], dtype=object)

In [8]:
# extract radial pos from the summary df
def average_radial_pos_from_summary_df (celltype_radial_pos_df):
    _cell_radial_pos_list=[]
    for _cell_idx in range(len(celltype_radial_pos_df)):
        _cell_radial_pos = celltype_radial_pos_df.iloc[_cell_idx]['radial_position']
        _cell_radial_pos_summary = pd.DataFrame(_cell_radial_pos.groupby(by='loci_name')['norm_radial_position'].mean())
        _cell_radial_pos_summary['uid'] = celltype_radial_pos_df.iloc[_cell_idx].name
        _cell_radial_pos_list.append(_cell_radial_pos_summary)

    celltype_radial_pos_summary=pd.concat(_cell_radial_pos_list)
    return celltype_radial_pos_summary

In [12]:
selected_cell_labels = ['L2/3 IT','L4/5 IT','L5 IT','L6 IT','L5 ET','L5/6 NP','L6 CT','L6b',
                           'Sst','Pvalb','Lamp5','Sncg','Vip',
                           'Astro','Oligo','OPC','Micro','Endo','VLMC','SMC','Peri', 
                           #'other',
                          ]

celltype_palette = {'Astro':'lightcoral', 
                    'Endo':'skyblue', 
                    'L2/3 IT':'gold', 
                    'L4/5 IT':'darkorange', 
                    'L5 ET':'mediumseagreen', 
                    'L5 IT':'aqua',
                    'L5/6 NP':'darkgreen',
                    'L6 CT':'brown',
                    'L6 IT':'magenta',
                    'L6b':'blue', 
                    'Lamp5':'orange', 
                    'Micro':'peachpuff',
                    'OPC':'thistle', 
                    'Oligo':'darkviolet',
                    'Peri':'sandybrown',
                    'Pvalb':'springgreen',
                    'SMC':'rosybrown',
                    'Sncg':'darkkhaki',
                    'Sst':'steelblue', 
                    'VLMC':'saddlebrown', 
                    'Vip':'red',
                    'other':'slategray'}


sel_merged_radial_pos_df = merged_radial_pos_df[merged_radial_pos_df['subclass'].isin(selected_cell_labels)]

#min_pts_th =1200
min_pts_th = 600
max_pts_th =3900
# sort plotting order by meadian of groups
df_plot = sel_merged_radial_pos_df[(sel_merged_radial_pos_df['total_spots']>=min_pts_th)&(sel_merged_radial_pos_df['total_spots']<max_pts_th)]
df_plot

,subclass,total_spots,radial_position
uid,,,
259202492748634617304623818845147108919,Oligo,863,loci_name fiber_id norm...
336074706103419484501535095917410831417,Oligo,1426,loci_name fiber_id norm...
179117357717369278884189379535888871018,Oligo,1601,loci_name fiber_id norm...
92271180002801802991585284299263797402,Oligo,2034,loci_name fiber_id norm...
237111112842863990529297306931448726772,Oligo,942,loci_name fiber_id norm...
...,...,...,...
245342798166567655308810294286385442992,VLMC,1613,loci_name fiber_id norm...
121463809145385827894569077562457697797,VLMC,913,loci_name fiber_id norm...
71119619609696733523981100580894538756,VLMC,734,loci_name fiber_id norm...


## Summarize

In [15]:
import tqdm
sorted_median_radial_pos_list = []
for _class_ind, _class in tqdm.tqdm(enumerate(selected_cell_labels)):
    #fig, ax = plt.subplots(figsize=(_single_col_width*3, _single_col_width), dpi=200)
    celltype_radial_pos_df = df_plot[df_plot['subclass']==_class]
    celltype_radial_pos_summary = average_radial_pos_from_summary_df (celltype_radial_pos_df)
    median_radial_pos = pd.DataFrame(celltype_radial_pos_summary.groupby(by='loci_name')['norm_radial_position'].median())
    sorted_median_radial_pos = codebook_df[['chr','chr_order']].copy(deep=True)
    sorted_median_radial_pos['norm_radial_position'] = median_radial_pos['norm_radial_position']
    sorted_median_radial_pos['total_cell'] = celltype_radial_pos_summary.groupby(by='loci_name')['norm_radial_position'].count()
    sorted_median_radial_pos['subclass'] = _class
    sorted_median_radial_pos_list.append(sorted_median_radial_pos)
    
sorted_median_radial_pos_all = pd.concat(sorted_median_radial_pos_list)
sorted_median_radial_pos_all

21it [03:50, 10.97s/it]


,chr,chr_order,norm_radial_position,total_cell,subclass
loci_name,,,,,
chr1_3742742_3759944,1,0,0.860136,1022.0,L2/3 IT
chr1_6245958_6258969,1,1,0.855358,969.0,L2/3 IT
chr1_8740008_8759916,1,2,0.850017,628.0,L2/3 IT
chr1_9627926_9637875,1,3,0.839728,960.0,L2/3 IT
chr1_9799472_9811359,1,4,0.826204,900.0,L2/3 IT
...,...,...,...,...,...
chrX_166247682_166259932,X,60,0.812004,137.0,Peri
chrX_167157164_167167452,X,61,0.734733,145.0,Peri
chrX_168746045_168757590,X,62,0.765202,134.0,Peri


## Save

In [16]:
output_fname = os.path.join(output_analysis_folder, f'radial_pos_convexhull_median_summary_{min_pts_th}pts_bysubclass_v2.csv')
sorted_median_radial_pos_all.to_csv(output_fname)